In [ ]:
path = '/data2/ccimmu/省肿瘤'
label_csv = None

def exists(val):
    return val is not None

pathid_orig = [f[:-6] for f in os.listdir(os.path.join(path, "store_4")) if not f.startswith(".")]
    # 【:-6】的原因：pyvips切割wsi后，生成的文件自动加上了后缀_files,正好是6个字节
    #pathid_orig = [f[:-6] for f in os.listdir(os.path.join(path, "pseudo_store_3")) if not f.startswith(".")]
    
if exists(label_csv):
    df_label = pd.read_csv(label_csv, index_col = "pathid")
pathid = []
images = []
labels = []
for i, id in enumerate(pathid_orig):
    if len(glob.glob(os.path.join(path, "store_4", str(id) + "_files","0", "*.jpeg"))) != 0:
        pathid.append(id)
        # 若生成的csv中的images为空缺，需检查图片的格式是否正确。例如，需要区分.jpeg和.jpg
        #重要提示：注意最后的文件后缀。images.append(glob.glob(os.path.join(path, "store_3", str(id) + "_files","0", "*.jpeg")))
        images.append(glob.glob(os.path.join(path, "store_4", str(id) + "_files","0", "*.jpeg")))
        if exists(label_csv):
            labels.append(df_label.loc[id]["labels"])
        else:
            labels.append(1e-6)
    else:
        pathid_orig_2 = [f for f in os.listdir(os.path.join(path, "store_4", str(id) + "_files")) if not f.startswith(".")]
        for j, id_2 in enumerate(pathid_orig_2):
            pathid.append(id_2)
            images.append(glob.glob(os.path.join(path, "store_4", str(id) + "_files", str(id_2), "0", "*.jpeg")))
            if exists(label_csv):
                labels.append(df_label.loc[id]["labels"])
            else:
                labels.append(1e-6)
c = {"pathid": pathid, "images": images, "labels": labels}
#df_temp = pd.DataFrame(c)
#df_temp.to_csv("files_label/store_4_temp.csv", index=False, encoding="utf_8_sig")
#torch.save(df.to_dict(orient='list'), "files_label/store_4_temp.db")
#df = pd.read_csv("files_label/store_4_temp.csv", index_col = "pathid")

In [ ]:
label_csv = "/data2/patho-vit_5_23_ccim/low/files_label/store_4_cc.csv"
if exists(label_csv):
    df_label = pd.read_csv(label_csv)

index_train, index_valid = train_test_split(range(len(df_label)), test_size = 0.2)
df_train = df_label.loc[index_train].reset_index(drop = True)
df_valid = df_label.loc[index_valid].reset_index(drop = True)

pathid = []
images = []
labels = []
for i, id in enumerate(df_train.pathid):
    pathid.append(id)
    #images.append(df.loc[id]["images"])
    index = c["pathid"].index(id)
    images.append(c["images"][index])
    labels.append(df_train.loc[i].labels)
c2 = {"pathid": pathid, "images": images, "labels": labels}
df_train = pd.DataFrame(c2) 

pathid = []
images = []
labels = []
for i, id in enumerate(df_valid.pathid):
    pathid.append(id)
    #images.append(df.loc[id]["images"])
    index = c["pathid"].index(id)
    images.append(c["images"][index])
    labels.append(df_valid.loc[i].labels)
c3 = {"pathid": pathid, "images": images, "labels": labels}
df_valid = pd.DataFrame(c3) 

df_train.to_csv("files_label/store_4_train_cc.csv", index=False, encoding="utf_8_sig")
torch.save(df_train.to_dict(orient='list'), "files_label/store_4_train_cc.db")

df_valid.to_csv("files_label/store_4_valid_cc.csv", index=False, encoding="utf_8_sig")
torch.save(df_valid.to_dict(orient='list'), "files_label/store_4_valid_cc.db")

In [1]:
import os
from sklearn.metrics import balanced_accuracy_score
import gc

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.transforms as transforms

from torch.optim.lr_scheduler import StepLR
import torch.backends.cudnn as cudnn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path = "/data2/patho-vit_5_23_ccsurv"
# 此处patho_vit后的_5_23为包的版本号
# 若此jupyternotebook运行中kernel挂掉，重启后仅需运行此一代码块，然后跳到需要运行的代码块即可

In [2]:
seed = 22

def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [3]:
# 第二部门：机场1——全面训练海绵宝宝
batch_size = 400
# 取决于gpu数量，原始文献为50，若报错CUDA OUT OF MEMORY,则减少batch_size个数。此外，需关闭kernel重启，只重启机场1代码即可。
num_workers = 60
# 取决于cpu数量，原始文献此处为4，可从0逐渐增加

epochs = 10000
# 原始文献中默认值为50
test_every = 1
# 多少次训练后，进行一次验证。原始文献中默认值为10

In [4]:
normalize = transforms.Normalize(mean=(0.485, 0.456, 0.406), 
                             std=(0.229, 0.224, 0.225))
transform = transforms.Compose([transforms.ToTensor(), normalize])

#train_lib_0 = "/data2/patho-vit_5_23_ccim/low/files_label/store_4_lung_train.db"
train_lib_1 = "/data2/patho-vit_5_23_ccsurv/files_label/store_4_train_ccsurv.db"
valid_lib = "/data2/patho-vit_5_23_ccsurv/files_label/store_4_valid_ccsurv.db"
valid_lib_1 = "/data2/patho-vit_5_23_ccsurv/SZL_exva/files/store_4.db"

from patho_vit.airport1 import pathovitdataset

#train_dataset_0 = pathovitdataset(train_lib_0, transform)
#train_loader_0 = torch.utils.data.DataLoader(
#    train_dataset_0,
#    batch_size = batch_size, shuffle = True,
#    num_workers = num_workers)

train_dataset_1 = pathovitdataset(train_lib_1, transform)
train_loader_1 = torch.utils.data.DataLoader(
    train_dataset_1,
    batch_size = batch_size, shuffle = True,
    num_workers = num_workers
)

valid_dataset = pathovitdataset(valid_lib, transform)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size = batch_size, shuffle = False,
    num_workers = num_workers
)

valid_dataset_1 = pathovitdataset(valid_lib_1, transform)
valid_loader_1 = torch.utils.data.DataLoader(
    valid_dataset_1,
    batch_size = batch_size, shuffle = False,
    num_workers = num_workers
)

In [5]:
from collections import OrderedDict
from patho_vit.vit_luci import ViT

In [6]:
vit = ViT(
    image_size = 384,
    patch_size = 16, 
    channels = 3,
    dim = 768,
    depth = 12,
    heads = 12,
    mlp_dim = 768 * 4,
    num_classes = 2
)

In [7]:
# 导入定制vit，加载目标编码器的权重
#weights = torch.load("/data2/patho-vit_5_23_oc/up/13/gpvit_weight_6_24_epoch_1.pt")

weights = torch.load("/data2/patho-vit_5_23_ccsurv/up/13/12.4/gpvit_weight_12_4_epoch_18.pt")

new_dict = OrderedDict()
for k, v in weights.items():
    if "module.target_encoder" in k:
        new_key = k[22:]
        new_dict[new_key] = v

vit.to(device)
vit.load_state_dict(new_dict, strict = False)
vit = nn.DataParallel(vit)

In [7]:
# 导入定制vit，加载目标编码器的权重
#weights = torch.load("/data2/patho-vit_5_23_oc/up/13/gpvit_weight_6_24_epoch_1.pt")

weights = torch.load("/data2/patho-vit_5_23_ccsurv/low/4/12.7/checkpoint_ccsurv_10.pth")

##new_dict = OrderedDict()
#for k, v in weights.items():
#    if "module.target_encoder" in k:
#        new_key = k[22:]
#        new_dict[new_key] = v

vit.to(device)
vit = nn.DataParallel(vit)
vit.load_state_dict(weights, strict = False)

<All keys matched successfully>

In [8]:
cudnn.benchmark = True

# 定义丢失函数和优化器
#w = torch.Tensor([0.22, 0.365, 0.415])  
w = torch.Tensor([0.1,0.9]) 
# 设定阴性和阳性的惩罚比例，这里是胡乱取的数
criterion = nn.CrossEntropyLoss(w).to(device)
optimizer = torch.optim.AdamW(vit.parameters(), lr=3e-4, weight_decay = 0.05, eps = 1e-4, betas = [0.9, 0.95])
#scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
# 原始文献lr=1e-4，
# lucidrains为3e-5，且使用了scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

# 生成输出文件
fconv = open(os.path.join("/data2/patho-vit_5_23_ccsurv/low/4/convergence.csv"), "w")
fconv.write("epoch, metric, value\n")
fconv.close()

# 正式训练和验证
# 此处杀bug的重要心得：若出现TypeError: cannot convert the series to <class 'float'>，
# 表示生成的pathids_train_1024.db，或pathids_test_1024.db里有空缺的标签值，可通过
# 同名的xlsx进行查看。此处发现“202017947.A3.bif”切片未生成标签值，遂直接删除.
#best_or = 1
best_ba = 0
#global best_or
global best_ba

#total_step_0 = len(train_loader_0)
total_step_1 =  len(train_loader_1)
# 开始训练和验证
for epoch in range(epochs):
    torch.cuda.empty_cache()
    epoch_accuracy_lung = 0
    epoch_loss_lung = 0
    epoch_accuracy = 0
    epoch_loss = 0
    
    #for i, (images, labels) in enumerate(train_loader_0):
        #torch.cuda.empty_cache()
    #    images = images.to(device)
    #    labels = labels.to(device)
    #    outputs, _ = vit(images)
    #    loss_lung = criterion(outputs, labels.long())
        
    #    optimizer.zero_grad()
    #    loss_lung.backward()
    #    optimizer.step()
        
    #    acc_lung = (outputs.argmax(dim=-1) == labels).float().mean()
    #    epoch_accuracy_lung += acc_lung / len(train_loader_0)
    #    epoch_loss_lung += loss_lung / len(train_loader_0)
        
    #    if (i + 1) % 2558 == 0:
        #if (i + 1) % 5230 == 0:
    #        print("Epoch [{}/{}], Step [{}/{}] Loss_lung: {:.10f}"
    #             .format(epoch+1, epochs, i+1, total_step_0, loss_lung))
            
    for i, (images, labels) in enumerate(train_loader_1):
        #torch.cuda.empty_cache()
        images = images.to(device)
        labels = labels.to(device)
        outputs, _ = vit(images)
        loss = criterion(outputs, labels.long())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = (outputs.argmax(dim=-1) == labels).float().mean()
        epoch_accuracy += acc / len(train_loader_1)
        epoch_loss += loss / len(train_loader_1)
        
        if (i + 1) % 2461 == 0:
        #if (i + 1) % 5230 == 0:
            print("Epoch [{}/{}], Step [{}/{}] Loss: {:.10f}"
                 .format(epoch+1, epochs, i+1, total_step_1, loss))
    
    print("Epoch [{}/{}], Loss_cc: {:.4f}, Acc_cc: {:.4f}".format(epoch+1, epochs, epoch_loss, epoch_accuracy))
    
    fconv = open(os.path.join("/data2/patho-vit_5_23_ccsurv/low/4/convergence.csv"), "a")
    #fconv.write("{}, loss_lung, {:.4f}\n".format(epoch+1, epoch_loss_lung))
    #fconv.write("{}, acc_lung, {:.4f}\n".format(epoch+1, epoch_accuracy_lung))
    fconv.write("{}, loss, {:.4f}\n".format(epoch+1, epoch_loss))
    fconv.write("{}, acc, {:.4f}\n".format(epoch+1, epoch_accuracy))
    fconv.close()
        
    if (epoch+1) % test_every == 0:
        with torch.no_grad():
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            
            pred = []
            real = []
            
            for images, labels in valid_loader:
                #torch.cuda.empty_cache()
                images = images.to(device)
                labels = labels.to(device)
                outputs, _ = vit(images)
                val_loss = criterion(outputs, labels.long())
                
                #outputs = outputs.softmax(dim = -1)
                outputs = outputs.argmax(dim = -1)
                
                acc = (outputs == labels).float().mean()
                epoch_val_accuracy += acc / len(valid_loader)
                epoch_val_loss += val_loss / len(valid_loader)
            
                pred.extend(list(outputs.cpu().numpy()))
                real.extend(list(labels.cpu().numpy()))
                
            #df_pred = pd.DataFrame(pred)
            #df_pred.to_csv("output_downstream/pred_real/pred_.csv")
            
            #df_real = pd.DataFrame(real)
            #df_real.to_csv("output_downstream/real.csv")
            
            pred = np.array(pred)
            real = np.array(real)
            ba = balanced_accuracy_score(real, pred)
            
            # del pred real
            # gc.collect()
            
            #c={"prediction": pred, "labels": real}
            #df_c = pd.DataFrame(c)
            #df_c.to_csv("output_jepa_label/pred_real/pred_{}.csv".format(epoch+1))
            
            #neq = np.not_equal(pred, real)
            #acc = 1 - float(neq.sum()) / pred.shape[0]
            
            eq = np.equal(pred, real)
            sensi = float(np.logical_and(pred==1, eq).sum()) / (real==1).sum()
            speci = float(np.logical_and(pred==0, eq).sum()) / (real==0).sum()
            
            #fpr = float(np.logical_and(pred==1, neq).sum()) / (real==0).sum()
            #fnr = float(np.logical_and(pred==0, neq).sum()) / (real==1).sum()
            
            #odds_ratio = (float(np.logical_and(pred==1, eq).sum()) * float(np.logical_and(pred==0, eq).sum())) \
            #            / (float(np.logical_and(pred==0, neq).sum()) * float(np.logical_and(pred==1, neq).sum()) + 1e-9)
            
            fconv = open(os.path.join("/data2/patho-vit_5_23_ccsurv/low/4/convergence.csv"), "a")
            fconv.write("{}, epoch_val_loss, {:.4f}\n".format(epoch+1, epoch_val_loss))
            fconv.write("{}, val_acc, {:.4f}\n".format(epoch+1, epoch_val_accuracy))
            fconv.write("{}, ba, {:.4f}\n".format(epoch+1, ba))
            fconv.write("{}, sensi, {:.4f}\n".format(epoch+1, sensi))
            fconv.write("{}, speci, {:.4f}\n".format(epoch+1, speci))
            #fconv.write("{}, odds_ratio, {:.4f}\n".format(epoch+1, odds_ratio))
            #fconv.write("{}, fpr, {}\n".format(epoch+1, fpr))
            #fconv.write("{}, fnr, {}\n".format(epoch+1, fnr))
            fconv.close()


            epoch_test_accuracy = 0
            #epoch_test_loss = 0
            
            pred_test = []
            real_test = []
            
            for images, labels in valid_loader_1:
                #torch.cuda.empty_cache()
                images = images.to(device)
                labels = labels.to(device)
                outputs, _ = vit(images)
                #test_loss = criterion(outputs, labels.long())
                
                #outputs = outputs.softmax(dim = -1)
                outputs = outputs.argmax(dim = -1)
                
                test_acc = (outputs == labels).float().mean()
                epoch_test_accuracy += test_acc / len(valid_loader_1)
                #epoch_test_loss += val_loss / len(valid_loader_1)
            
                pred_test.extend(list(outputs.cpu().numpy()))
                real_test.extend(list(labels.cpu().numpy()))
                
            #df_pred = pd.DataFrame(pred)
            #df_pred.to_csv("output_downstream/pred_real/pred_.csv")
            
            #df_real = pd.DataFrame(real)
            #df_real.to_csv("output_downstream/real.csv")
            
            pred_test = np.array(pred_test)
            real_test = np.array(real_test)
            ba_test = balanced_accuracy_score(real_test, pred_test)
            
            # del pred real
            # gc.collect()
            
            #c={"prediction": pred, "labels": real}
            #df_c = pd.DataFrame(c)
            #df_c.to_csv("output_jepa_label/pred_real/pred_{}.csv".format(epoch+1))
            
            #neq = np.not_equal(pred, real)
            #acc = 1 - float(neq.sum()) / pred.shape[0]
            
            eq_test = np.equal(pred_test, real_test)
            sensi_test = float(np.logical_and(pred_test==1, eq_test).sum()) / (real_test==1).sum()
            speci_test = float(np.logical_and(pred_test==0, eq_test).sum()) / (real_test==0).sum()
            
            #fpr = float(np.logical_and(pred==1, neq).sum()) / (real==0).sum()
            #fnr = float(np.logical_and(pred==0, neq).sum()) / (real==1).sum()
            
            #odds_ratio = (float(np.logical_and(pred==1, eq).sum()) * float(np.logical_and(pred==0, eq).sum())) \
            #            / (float(np.logical_and(pred==0, neq).sum()) * float(np.logical_and(pred==1, neq).sum()) + 1e-9)
            
            fconv = open(os.path.join("/data2/patho-vit_5_23_ccsurv/low/4/convergence.csv"), "a")
            #fconv.write("{}, epoch_val_loss, {:.4f}\n".format(epoch+1, epoch_val_loss))
            fconv.write("{}, test_acc, {:.4f}\n".format(epoch+1, epoch_test_accuracy))
            fconv.write("{}, ba_test, {:.4f}\n".format(epoch+1, ba_test))
            fconv.write("{}, sensi_test, {:.4f}\n".format(epoch+1, sensi_test))
            fconv.write("{}, speci_test, {:.4f}\n".format(epoch+1, speci_test))
            #fconv.write("{}, odds_ratio, {:.4f}\n".format(epoch+1, odds_ratio))
            #fconv.write("{}, fpr, {}\n".format(epoch+1, fpr))
            #fconv.write("{}, fnr, {}\n".format(epoch+1, fnr))
            fconv.close()
            
    torch.save(vit.state_dict(), os.path.join("/data2/patho-vit_5_23_ccsurv/low/4/12.11/checkpoint_ccsurv_{}.pth".format(epoch+1)))
        
    #if ba >= best_ba:
        # 或用 if ba >= best_ba:
        #    best_or = odds_ratio
    #    best_ba = ba
        
    #torch.save(vit.state_dict(), os.path.join("/data2/patho-vit_5_23_ccim/low/4-13waibu/best_checkpoint_2classes_{}.pth".format(epoch+1)))

    print(
        f"Epoch : {epoch+1} - val_loss: {epoch_val_loss:.4f}, val_acc: {epoch_val_accuracy:.4f} - ba: {ba:.4f}\n" 
    )
    print(
        f"Epoch : {epoch+1} - test_acc: {epoch_test_accuracy:.4f} - ba_test: {ba_test:.4f}\n" 
    )

Epoch [1/10000], Step [2461/11392] Loss: 0.3547721803
Epoch [1/10000], Step [4922/11392] Loss: 0.4001185298
Epoch [1/10000], Step [7383/11392] Loss: 0.3311749101
Epoch [1/10000], Step [9844/11392] Loss: 0.3454498947
Epoch [1/10000], Loss_cc: 0.3716, Acc_cc: 0.8080
Epoch : 1 - val_loss: 0.7607, val_acc: 0.6671 - ba: 0.5111

Epoch : 1 - test_acc: 0.7985 - ba_test: 0.5203

Epoch [2/10000], Step [2461/11392] Loss: 0.3131391406
Epoch [2/10000], Step [4922/11392] Loss: 0.4424652755
Epoch [2/10000], Step [7383/11392] Loss: 0.3830794394
Epoch [2/10000], Step [9844/11392] Loss: 0.3443254232
Epoch [2/10000], Loss_cc: 0.3600, Acc_cc: 0.8152
Epoch : 2 - val_loss: 0.7662, val_acc: 0.6748 - ba: 0.5046

Epoch : 2 - test_acc: 0.7896 - ba_test: 0.5275

Epoch [3/10000], Step [2461/11392] Loss: 0.3624217510
Epoch [3/10000], Step [4922/11392] Loss: 0.3479746878
Epoch [3/10000], Step [7383/11392] Loss: 0.2962770164
Epoch [3/10000], Step [9844/11392] Loss: 0.2884314358
Epoch [3/10000], Loss_cc: 0.3497, Acc_

KeyboardInterrupt: 

In [ ]:
# 机场2，为每个补丁抽出特征，为整张切片制作特征图像

In [1]:
import os
from sklearn.metrics import balanced_accuracy_score
import gc

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.transforms as transforms

from torch.optim.lr_scheduler import StepLR
import torch.backends.cudnn as cudnn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path = "/data2/patho-vit_5_23_ccsurv/low"
# 此处patho_vit后的_5_23为包的版本号
# 若此jupyternotebook运行中kernel挂掉，重启后仅需运行此一代码块，然后跳到需要运行的代码块即可

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from PIL import Image
import cv2
import pandas as pd
import os

In [3]:
from patho_vit.vit_luci2 import ViT as ViT
from patho_vit.airport2 import train_features_extractor_gpvit2 as Extractor
from patho_vit.airport2 import valid_features_extractor_gpvit2 as Extractor2
from collections import OrderedDict


In [4]:
def Extractor3(path, train_lib, transform, batch_size, model, device):
    #model = model.to(device)
    #model = nn.DataParallel(model)
    model.eval()

    for i, image in enumerate(train_lib["images"]):
        cols = []
        rows=[]

        for patch in train_lib["images"][i]:
            col, row = patch.split("/")[-1].split(".")[0].split("_")
            cols.append(int(col))
            rows.append(int(row))
        col_last = sorted(np.array(cols))[-1]
        row_last = sorted(np.array(rows))[-1]
        image = torch.zeros((row_last+1), (col_last+1), 768)
    
        patches_batch = []  # 摆渡车
        patches_pos = []  # 补丁位置
        img_features =[]
        for patch in train_lib["images"][i]:
            col, row = patch.split("/")[-1].split(".")[0].split("_")
            patches_pos.append([int(col), int(row)])
    
        for j, patch in enumerate(train_lib["images"][i]):
            torch.cuda.empty_cache() 
            #patch = cv2.imread(patch)
            #patch = cv2.resize(patch, (384, 384))
            #patch = cv2.cvtColor(patch, cv2.COLOR_BGR2RGB)
            patch = Image.open(patch)

            patch = transform(patch).unsqueeze(0) 
            patches_batch.append(patch)
        
            if ((j+1) % batch_size == 0) or ((j+1) == len(train_lib["images"][i])):
                torch.cuda.empty_cache()  
                patches_batch = torch.cat(patches_batch, 0).to(device)
                _, features = model(patches_batch)
                #features = features.softmax(dim = -1)
                features = features.cpu().tolist()
                for k in range(len(features)):
                    img_features.append(features[k])
                patches_batch = []  # 清空摆渡车
    
        for index in range(len(patches_pos)):
            batch_range = torch.arange(patches_pos[index][1], patches_pos[index][1] + 1)[:, None]
            indices = torch.arange(patches_pos[index][0], patches_pos[index][0] + 1)
            image[batch_range, indices] = torch.Tensor(img_features[index]).reshape(1, 1, 768)
    
        #train_lib["images"][i] = image

        torch.save(image, "store_5_96/{}.db".format(train_lib["pathid"][i]))

    images = []
    for i, pathid in enumerate(train_lib["pathid"]):
        images.append(os.path.join(path, "store_5_96", pathid+".db"))
    c = {"pathid": train_lib["pathid"], "images": images, "labels": train_lib["labels"]}
    df = pd.DataFrame(c)
    df.to_csv("files_label/store_5_96_exva.csv", index=False, encoding="utf_8_sig")
    torch.save(c, "files_label/store_5_96_exva.db")
    return "store_5_96_exva.db"

In [5]:
vit = ViT(
    image_size = 384,
    patch_size = 16,
    dim = 768,
    depth = 12,
    heads = 12,
    mlp_dim = 768 * 4,
    num_classes = 2
)
#output, feature = vit(torch.randn(2, 3, 384, 384))
#feature.shape

In [6]:
# 导入定制vit，加载目标编码器的权重

#weights = torch.load("output_jepa/gpvit_weight_4_22_epoch_1.pt")
weights = torch.load("/data2/patho-vit_5_23_ccsurv/low/4/12.7/checkpoint_ccsurv_2.pth")

vit.to(device)
vit = nn.DataParallel(vit)
vit.load_state_dict(weights, strict = True)

<All keys matched successfully>

In [7]:
# 一个文件夹一个文件夹的抽特征，并重构（384，384，768）的图像
transform = transforms.Compose([
    #transforms.Resize([384, 384]),
    transforms.ToTensor(), 
    transforms.Normalize(mean=(0.485, 0.456, 0.406), 
                             std=(0.229, 0.224, 0.225))])

#train_lib = "files/store_5_train_{}.db".format((n+1) * 25)
train_lib = torch.load("/data2/patho-vit_5_23_ccsurv/files_label/store_4_train_ccsurv.db")
valid_lib = torch.load("/data2/patho-vit_5_23_ccsurv/files_label/store_4_valid_ccsurv.db")
exva_lib = torch.load("/data2/patho-vit_5_23_ccsurv/SZL_exva/files/store_4.db")

extractor = Extractor(path = path, train_lib = train_lib, transform = transform, batch_size = 200, model = vit, device = device)
extractor2 = Extractor2(path = path, train_lib = valid_lib, transform = transform, batch_size = 200, model = vit, device = device)
extractor3 = Extractor3(path = path, train_lib = exva_lib, transform = transform, batch_size = 200, model = vit, device = device)